In [7]:
import pandas as pd
import numpy as np
import sklearn.metrics as sklm

In [9]:
Features = np.array(pd.read_csv('Credit_Features.csv'))
Labels  = np.array(pd.read_csv('Credit_Labels.csv'))

Labels = Labels.reshape(Labels.shape[0])

In [12]:
import numpy.random as nr
Features_0 =  Features[Labels == 0,:] 
Features_1 =  Features[Labels == 1,:]
Labels_0   =  Labels[Labels == 0]
Labels_1   =  Labels[Labels == 1]
indx       =  nr.choice(Features_0.shape[0],Features_1.shape[0],replace = True)
temp_Labels   = np.concatenate((Labels_1,Labels_0[indx]),axis = 0)
temp_Features = np.concatenate((Features_1,Features_0[indx,:]),axis=0)

In [13]:
print(temp_Features)

[[ 1.7083687   1.16304581 -0.87018333 ...,  0.          1.          0.        ]
 [ 0.51728944  0.90474274  0.02414692 ...,  0.          0.          1.        ]
 [ 0.90073132  0.99762148  0.91847717 ...,  1.          0.          0.        ]
 ..., 
 [-0.67378981 -0.1815588  -0.87018333 ...,  0.          1.          0.        ]
 [ 0.28783389  0.03312571  0.91847717 ...,  0.          1.          0.        ]
 [-0.29034793  0.14853305 -0.87018333 ...,  0.          0.          0.        ]]


In [15]:
import sklearn.model_selection as ms
nr.seed(123)
inside = ms.KFold(n_splits = 10, shuffle = True)

nr.seed(99)
outside = ms.KFold(n_splits = 10, shuffle = True)

In [22]:
param_grid = {'learning_rate':[1,10,0.1]}

from sklearn.ensemble import AdaBoostClassifier
nr.seed(0)
estimator = AdaBoostClassifier()
estimator =  ms.GridSearchCV(estimator = estimator, cv = inside, param_grid = param_grid, scoring = 'roc_auc', 
                                return_train_score = True)

estimator.fit(temp_Features,temp_Labels)
print(estimator.best_estim,ator_.learning_rate)

0.1


In [23]:
nr.seed(9999)
estimate = ms.cross_val_score(estimator, temp_Features, temp_Labels, cv = outside)

In [27]:
print ("Mean of Performance Metirc = %4.3f"%np.mean(estimate))
print("Std of Performance Metric = %4.3f"%np.std(estimate))

Mean of Performance Metirc = 0.769
Std of Performance Metric = 0.043


In [30]:
for i,x in enumerate(estimate):
    print("Fold %2d   %4.3f"%(i+1,x))

Fold  1   0.821
Fold  2   0.792
Fold  3   0.781
Fold  4   0.687
Fold  5   0.765
Fold  6   0.752
Fold  7   0.737
Fold  8   0.778
Fold  9   0.730
Fold 10   0.845


In [35]:
nr.seed(777)
indx = range(temp_Features.shape[0])
print(temp_Features.shape[0])

600


In [36]:
nr.seed(777)
indx = ms.train_test_split(indx, test_size = 300 )
train_X = temp_Features[indx[0],:]
test_X  = temp_Features[indx[1],:]
train_y = np.ravel(temp_Labels[indx[0]])
test_y  = np.ravel(temp_Labels[indx[1]])

model = AdaBoostClassifier(learning_rate = estimator.best_estimator_.learning_rate)

In [37]:
model.fit(train_X,train_y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.1, n_estimators=50, random_state=None)

In [41]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])

def print_metrics(labels, probs, threshold):
    scores = score_model(probs, threshold)
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    print('AUC             %0.2f' % sklm.roc_auc_score(labels, probs[:,1]))
    print('Macro precision %0.2f' % float((float(metrics[0][0]) + float(metrics[0][1]))/2.0))
    print('Macro recall    %0.2f' % float((float(metrics[1][0]) + float(metrics[1][1]))/2.0))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])
    
probabilities = model.predict_proba(test_X)
print_metrics(test_y, probabilities, 0.5)    

                 Confusion matrix
                 Score positive    Score negative
Actual positive        92                58
Actual negative        32               118

Accuracy        0.70
AUC             0.75
Macro precision 0.71
Macro recall    0.70
 
           Positive      Negative
Num case      150           150
Precision    0.74          0.67
Recall       0.61          0.79
F1           0.67          0.72
